In [1]:
import requests
import json
from dotenv import load_dotenv
import os


In [2]:
import os
import requests
from dotenv import load_dotenv

# Load token from .env
load_dotenv()
FIGMA_TOKEN = os.getenv("FIGMA_TOKEN")
FIGMA_FILE_KEY = os.getenv("FIGMA_FILE_KEY")

if not FIGMA_TOKEN or not FIGMA_FILE_KEY:
    raise ValueError("Missing FIGMA_TOKEN or FIGMA_FILE_KEY in .env")

# Figma API endpoint
url = "https://api.figma.com/v1/files/ITjMkPhuORNbvS1GvHMba6"

headers = {
    "X-Figma-Token": FIGMA_TOKEN
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    print("Top-level keys in the Figma file:")
    print(data.keys())
    # Optionally save to a file
    with open("figma_design.json", "w", encoding="utf-8") as f:
        f.write(response.text)
else:
    print(f"Error {response.status_code}: {response.text}")


Top-level keys in the Figma file:
dict_keys(['document', 'components', 'componentSets', 'schemaVersion', 'styles', 'name', 'lastModified', 'thumbnailUrl', 'version', 'role', 'editorType', 'linkAccess'])


In [3]:

with open("figma_design.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def walk(node, depth=0):
    name = node.get("name")
    ntype = node.get("type")
    if ntype == "FRAME":
        print("📱 Frame:", name)
    elif ntype == "TEXT":
        chars = node.get("characters", "")
        print("   " * depth + f"🅰️ Text: {name} -> {chars}")
    elif ntype in ("COMPONENT", "INSTANCE", "RECTANGLE"):
        print("   " * depth + f"🔘 Element: {name} ({ntype})")
    for child in node.get("children", []):
        walk(child, depth + 1)

walk(data["document"])


      🔘 Element: Button (INSTANCE)
         🔘 Element: Star (INSTANCE)
         🅰️ Text: Button -> Login
         🔘 Element: X (INSTANCE)
      🔘 Element: Textarea Field (INSTANCE)
         🅰️ Text: Label -> User name
         🅰️ Text: Description -> Description
📱 Frame: Textarea
            🅰️ Text: Value -> 
         🅰️ Text: Hint -> Hint
      🔘 Element: Textarea Field (INSTANCE)
         🅰️ Text: Label -> Password
         🅰️ Text: Description -> Description
📱 Frame: Textarea
            🅰️ Text: Value -> 
         🅰️ Text: Hint -> Hint


In [4]:
elements_by_screen = {}

def extract_text_from_node(node, depth=0):
    texts = []
    ntype = node.get("type")
    name = node.get("name")
    
    # Print for tracing
    print("  " * depth + f"Visiting node: type={ntype}, name={name}")
    
    if ntype == "TEXT" and "characters" in node:
        print("  " * depth + f"  Found text: {node['characters']}")
        texts.append(node["characters"])
    
    for child in node.get("children", []):
        texts.extend(extract_text_from_node(child, depth + 1))
    return texts

def extract_elements(node, current_screen=None, depth=0):
    ntype = node.get("type")
    name = node.get("name")
    
    print("  " * depth + f"Node type={ntype}, name={name}, current_screen={current_screen}")
    
    # FRAME becomes a new screen
    if ntype == "FRAME":
        current_screen = name
        elements_by_screen[current_screen] = []
        print("  " * depth + f"--> New frame detected: {current_screen}")
    
    # If current_screen is None, fallback to CANVAS or DOCUMENT
    if current_screen is None:
        if ntype in ("CANVAS", "DOCUMENT"):
            current_screen = name
            elements_by_screen[current_screen] = []
            print("  " * depth + f"--> Using fallback screen: {current_screen}")
    
    # Capture INSTANCE, COMPONENT, RECTANGLE under the current screen
    if current_screen and ntype in ("INSTANCE", "COMPONENT", "RECTANGLE"):
        text_layers = extract_text_from_node(node, depth + 1)
        if text_layers:
            print("  " * depth + f"--> Adding element: {name}, text: {text_layers}")
            elements_by_screen[current_screen].append({"type": name, "text": text_layers})
    
    # Recurse into children
    for child in node.get("children", []):
        extract_elements(child, current_screen, depth + 1)

extract_elements(data["document"])
print("\nFinal elements_by_screen:")
print(elements_by_screen)


Node type=DOCUMENT, name=Document, current_screen=None
--> Using fallback screen: Document
  Node type=CANVAS, name=Page 1, current_screen=Document
    Node type=INSTANCE, name=Button, current_screen=Document
      Visiting node: type=INSTANCE, name=Button
        Visiting node: type=INSTANCE, name=Star
          Visiting node: type=VECTOR, name=Icon
        Visiting node: type=TEXT, name=Button
          Found text: Login
        Visiting node: type=INSTANCE, name=X
          Visiting node: type=VECTOR, name=Icon
    --> Adding element: Button, text: ['Login']
      Node type=INSTANCE, name=Star, current_screen=Document
        Visiting node: type=INSTANCE, name=Star
          Visiting node: type=VECTOR, name=Icon
        Node type=VECTOR, name=Icon, current_screen=Document
      Node type=TEXT, name=Button, current_screen=Document
      Node type=INSTANCE, name=X, current_screen=Document
        Visiting node: type=INSTANCE, name=X
          Visiting node: type=VECTOR, name=Icon
    

In [5]:
import json

def generate_test_cases(elements_by_screen):
    """
    Generate step-by-step QA test cases for each screen based on extracted elements.
    Returns a Markdown string with all test cases.
    """
    md = "# Automated QA Test Cases\n\n"
    
    for screen, elements in elements_by_screen.items():
        md += f"## Screen: {screen}\n\n"
        test_counter = 1
        
        # Identify inputs and buttons
        inputs = [el for el in elements if "text" in el and any(kw in el["type"].lower() for kw in ["textarea", "input", "field"])]
        buttons = [el for el in elements if "text" in el and "button" in el["type"].lower()]
        
        # Generate input tests
        for inp in inputs:
            label_texts = [txt for txt in inp["text"] if txt.strip()]
            label = label_texts[0] if label_texts else inp["type"]
            
            # Empty input
            md += f"### Test Case {test_counter}: Empty {label}\n"
            md += f"1. Leave the '{label}' field empty.\n"
            md += "2. Click submit / proceed (if any button present).\n"
            md += f"3. Expect: Error message indicating '{label}' is required.\n\n"
            test_counter += 1
            
            # Valid input
            md += f"### Test Case {test_counter}: Valid {label}\n"
            md += f"1. Enter a valid value in the '{label}' field.\n"
            md += "2. Click submit / proceed (if any button present).\n"
            md += "3. Expect: Field accepted, continue to next step.\n\n"
            test_counter += 1
            
            # Invalid input
            md += f"### Test Case {test_counter}: Invalid {label}\n"
            md += f"1. Enter an invalid value in the '{label}' field.\n"
            md += "2. Click submit / proceed.\n"
            md += f"3. Expect: Error message indicating '{label}' input invalid.\n\n"
            test_counter += 1
        
        # Generate button tests
        for btn in buttons:
            btn_texts = [txt for txt in btn["text"] if txt.strip()]
            btn_label = btn_texts[0] if btn_texts else btn["type"]
            md += f"### Test Case {test_counter}: Click '{btn_label}' Button\n"
            md += f"1. Click the '{btn_label}' button.\n"
            md += "2. Expect: Appropriate action triggered (navigation, submit, or modal).\n\n"
            test_counter += 1
            
    return md

# Example usage
md_test_cases = generate_test_cases(elements_by_screen)

# Optionally, save to file
with open("qa_test_cases.md", "w", encoding="utf-8") as f:
    f.write(md_test_cases)

print("✅ Test cases generated successfully. Preview:\n")
print(md_test_cases[:1000])  # print first 1000 chars


✅ Test cases generated successfully. Preview:

# Automated QA Test Cases

## Screen: Document

### Test Case 1: Empty User name
1. Leave the 'User name' field empty.
2. Click submit / proceed (if any button present).
3. Expect: Error message indicating 'User name' is required.

### Test Case 2: Valid User name
1. Enter a valid value in the 'User name' field.
2. Click submit / proceed (if any button present).
3. Expect: Field accepted, continue to next step.

### Test Case 3: Invalid User name
1. Enter an invalid value in the 'User name' field.
2. Click submit / proceed.
3. Expect: Error message indicating 'User name' input invalid.

### Test Case 4: Empty Password
1. Leave the 'Password' field empty.
2. Click submit / proceed (if any button present).
3. Expect: Error message indicating 'Password' is required.

### Test Case 5: Valid Password
1. Enter a valid value in the 'Password' field.
2. Click submit / proceed (if any button present).
3. Expect: Field accepted, continue to next ste

In [ ]:
import torch
from google import genai

from transformers import AutoModelForCausalLM, AutoTokenizer

# ---------------------------
# 1. Prepare the prompt for the LLM
# ---------------------------

client = genai.Client()


# Construct a textual prompt for the LLM
prompt = "Generate detailed QA test cases (step-by-step) for the following screens and elements:\n\n"

for screen, elements in elements_by_screen.items():
    prompt += f"Screen: {screen}\n"
    for elem in elements:
        prompt += f"- {elem['type']} with labels: {', '.join(elem['text'])}\n"
prompt += "\nOutput each test case in step-by-step format."

print("Prompt sent to LLM:\n", prompt)

# ---------------------------
# 2. Load a Hugging Face causal LM (local, free)
# ---------------------------

model_name = "HuggingFaceTB/SmolLM-1.7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

# ---------------------------
# 3. Encode prompt and generate
# ---------------------------

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

# Decode generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n=== Generated Test Cases ===\n")
print(generated_text)


Prompt sent to LLM:
 Generate detailed QA test cases (step-by-step) for the following screens and elements:

Screen: Document
- Button with labels: Login
- Textarea Field with labels: User name, Description, , Hint
- Textarea Field with labels: Password, Description, , Hint
Screen: Textarea

Output each test case in step-by-step format.


d:\venvs\Agents-env\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\huggingface_cache\hub\models--HuggingFaceTB--SmolLM-1.7B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For bet

KeyboardInterrupt: 

In [10]:
from google import genai

client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)


ImportError: cannot import name 'genai' from 'google' (unknown location)

In [10]:
from google import genai
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

# Get your API key
GEMINI_KEY = os.getenv("GEMINI_KEY")

# Initialize the client
client = genai.Client(api_key=GEMINI_KEY)

# Generate content
response = client.models.generate_content(
    model="gemini-2.5-flash",           # model name is correct
    contents="Explain how AI works in a few words",  # input prompt
)

# Print the output
print(response.text)


ValueError: Missing key inputs argument! To use the Google AI API, provide (`api_key`) arguments. To use the Google Cloud API, provide (`vertexai`, `project` & `location`) arguments.